In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from pathlib import Path

from torch.utils.data import DataLoader

from src.dataset.dataset import FurnitureStateDataset

from src.dataset.dataloader import EndlessDataloader

import numpy as np

from src.visualization.render_mp4 import create_mp4_jupyter

In [7]:
def get_demo_data_loader(control_mode, batch_size, num_workers=4) -> DataLoader:
    demo_data = FurnitureStateDataset(
        # dataset_paths=Path(cfg.data_path[0]),
        # pred_horizon=cfg.data.pred_horizon,
        # obs_horizon=cfg.data.obs_horizon,
        # action_horizon=cfg.data.action_horizon,
        # normalizer=normalizer,
        # data_subset=cfg.data.data_subset,
        # control_mode=cfg.control.control_mode,
        # first_action_idx=cfg.actor.first_action_index,
        # pad_after=cfg.data.get("pad_after", True),
        # max_episode_count=cfg.data.get("max_episode_count", None),
        dataset_paths=Path(
            "/data/scratch/ankile/furniture-data/processed/sim/one_leg/teleop/low/success.zarr"
        ),
        pred_horizon=1,
        obs_horizon=1,
        action_horizon=1,
        normalizer=None,
        data_subset=None,
        control_mode=control_mode,
        first_action_idx=0,
        pad_after=False,
        max_episode_count=None,
    )

    demo_data_loader = EndlessDataloader(
        dataset=demo_data,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=True,
        drop_last=False,
        persistent_workers=False,
    )

    return demo_data_loader


In [11]:
dataset = get_demo_data_loader("delta", 32).dataset

Loading zarr files:   0%|          | 0/1 [00:00<?, ?it/s]








































Loading zarr files: 100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


Loading dataset of 50 episodes:
  ('one_leg', 'teleop', 'low', 'success'): 50 episodes, 29020


In [21]:
# Get the episode data
ep_ends = dataset.episode_ends

images_list = np.split(dataset.train_data["color_image2"], ep_ends[:-1])
images_list = [image.numpy() for image in images_list]

In [26]:
for i, images in enumerate(images_list[:]):
    create_mp4_jupyter(images, f"demo_{i}.mp4", fps=20)

File saved as demo_0.mp4


File saved as demo_1.mp4


File saved as demo_2.mp4


File saved as demo_3.mp4


File saved as demo_4.mp4


...

File saved as demo_45.mp4


File saved as demo_46.mp4


File saved as demo_47.mp4


File saved as demo_48.mp4


File saved as demo_49.mp4
